# Optimization Project
## Nidhish Nerur & Sandra You

## Part 1: Data Preprocessing for Candidate Data

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#Read in the candidate dataset
df_emp = pd.read_csv("employee_data.csv")

#Read in the job dataset
df_job = pd.read_csv("job_descriptions.csv")

In [3]:
df_job.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [15]:
df_job['Job Title'][826]

'Investment Banker'

In [90]:
#Find years worked using start and exit dates
#Parse the StartDate and ExitDate columns
df_emp['StartDate'] = pd.to_datetime(df_emp['StartDate'], format='%d-%b-%y', errors='coerce')
df_emp['ExitDate'] = pd.to_datetime(df_emp['ExitDate'], format='%d-%b-%y', errors='coerce')

#Get the current date
current_date = pd.Timestamp(datetime.now())

#Calculate YearsWorked
def calculate_years_worked(row):
    start_date = row['StartDate']
    exit_date = row['ExitDate']
    if pd.isnull(exit_date):  # If ExitDate is missing
        return (current_date - start_date).days / 365.0
    else:
        return (exit_date - start_date).days / 365.0

df_emp['YearsWorked'] = df_emp.apply(calculate_years_worked, axis=1)

In [91]:
#Map job titles to numbers
#List of titles from data
titles = df_emp['Title'].value_counts().index.to_list()

#Create the mapping
title_mapping = {title: idx for idx, title in enumerate(titles)}

df_emp['TitleMapped'] = df_emp['Title'].map(title_mapping)

In [92]:
#Create proxy for education level
#Degree mapping estimates
degree_mapping = {
    'Production Technician I': "Bachelors",
    'Production Technician II': "Bachelors",
    'Area Sales Manager': "Bachelors",
    'Production Manager': "Bachelors",
    'Software Engineer': "Bachelors",
    'IT Support': "Bachelors",
    'Network Engineer': "Bachelors",
    'Sr. Network Engineer': "Bachelors",
    'Data Analyst': "Bachelors",
    'BI Developer': "Bachelors",
    'Administrative Assistant': "Bachelors",
    'Accountant I': "Bachelors",
    'Sales Manager': "Bachelors",
    'Senior BI Developer': "Masters",
    'Database Administrator': "Bachelors",
    'IT Manager - DB': "Masters",
    'Shared Services Manager': "Bachelors",
    'Sr. Accountant': "Masters",
    'IT Manager - Support': "Masters",
    'Director of Sales': "Masters",
    'Director of Operations': "Masters",
    'IT Manager - Infra': "Masters",
    'IT Director': "Masters",
    'President & CEO': "Masters",
    'BI Director': "Masters",
    'CIO': "Masters",
    'Software Engineering Manager': "Masters",
    'Data Analyst ': "Bachelors",
    'Sr. DBA': "Masters",
    'Principal Data Architect': "Masters",
    'Data Architect': "Masters",
    'Enterprise Architect': "PhD"
}

#Add the 'Education' column based on the Title
df_emp['Education'] = df_emp['Title'].map(degree_mapping)

#Map education levels to numeric values
education_mapping = {"Bachelors": 0, "Masters": 1, "PhD": 2}
df_emp['EducationMapped'] = df_emp['Education'].map(education_mapping)

In [93]:
#Map business units to numbers
#Extract unique business units from value_counts
unique_business_units = df_emp['BusinessUnit'].value_counts().index.tolist()

#Create the mapping
business_unit_mapping = {unit: idx for idx, unit in enumerate(unique_business_units)}

#Map the business units to numeric values in the DataFrame
df_emp['BusinessUnitMapped'] = df_emp['BusinessUnit'].map(business_unit_mapping)


In [94]:
#Map employee status to numbers
#Extract unique employee statuses from value_counts
unique_employee_statuses = df_emp['EmployeeStatus'].value_counts().index.tolist()

#Create the mapping
employee_status_mapping = {status: idx for idx, status in enumerate(unique_employee_statuses)}

#Map the employee statuses to numeric values in the DataFrame
df_emp['EmployeeStatusMapped'] = df_emp['EmployeeStatus'].map(employee_status_mapping)

In [95]:
#Map employee types to numbers
#Extract unique employee types from value_counts
unique_employee_types = df_emp['EmployeeType'].value_counts().index.tolist()

#Create the mapping
employee_type_mapping = {etype: idx for idx, etype in enumerate(unique_employee_types)}

#Map the employee types to numeric values in the DataFrame
df_emp['EmployeeTypeMapped'] = df_emp['EmployeeType'].map(employee_type_mapping)


In [96]:
#Adjust the pay zone from categories to numeric values
#Set seed for reproducibility
np.random.seed(123)

#Function to map PayZone to random float values
def map_payzone_to_random(zone):
    if zone == "Zone A":
        return np.random.uniform(130000, 200000)
    elif zone == "Zone B":
        return np.random.uniform(70000, 130000)
    elif zone == "Zone C":
        return np.random.uniform(10000, 70000)
    else:
        return np.nan  #Handle unexpected values

#Apply the mapping to the PayZone column
df_emp['PayZoneMapped'] = df_emp['PayZone'].apply(map_payzone_to_random)

In [97]:
#Map employee classification type to numbers
#Extract unique class types from value_counts
unique_employee_class_types = df_emp['EmployeeClassificationType'].value_counts().index.tolist()

#Create the mapping
class_type_mapping = {etype: idx for idx, etype in enumerate(unique_employee_class_types)}

#Map the class types to numeric values in the DataFrame
df_emp['EmployeeClassTypeMapped'] = df_emp['EmployeeClassificationType'].map(class_type_mapping)

In [98]:
#Map termination type to numbers
#Extract unique termination types from value_counts
unique_employee_term_types = df_emp['TerminationType'].value_counts().index.tolist()

#Create the mapping
term_type_mapping = {etype: idx for idx, etype in enumerate(unique_employee_term_types)}

#Map the termination types to numeric values in the DataFrame
df_emp['TerminationMapped'] = df_emp['TerminationType'].map(term_type_mapping)

In [99]:
#Map department type to numbers
#Extract unique department types from value_counts
unique_dep_types = df_emp['DepartmentType'].value_counts().index.tolist()

#Create the mapping
dep_type_mapping = {etype: idx for idx, etype in enumerate(unique_dep_types)}

#Map the department types to numeric values in the DataFrame
df_emp['DepartmentMapped'] = df_emp['DepartmentType'].map(dep_type_mapping)

In [100]:
#Map division type to numbers
#Extract unique division types from value_counts
unique_division_types = df_emp['Division'].value_counts().index.tolist()

#Create the mapping
div_type_mapping = {etype: idx for idx, etype in enumerate(unique_division_types)}

#Map the division types to numeric values in the DataFrame
df_emp['DivisionMapped'] = df_emp['Division'].map(div_type_mapping)

In [101]:
#Cast DOB to datetime
df_emp['DOB'] = pd.to_datetime(df_emp['DOB'], format='%d-%m-%Y', errors='coerce')

#Calculate age in years by subtracting from today's date
current_date = datetime.now()
#Floor division to get integer years
df_emp['Age'] = (current_date - df_emp['DOB']).dt.days // 365  

In [102]:
#Map gender to numbers
#Extract unique gender types from value_counts
unique_gender_types = df_emp['GenderCode'].value_counts().index.tolist()

#Create the mapping
gender_type_mapping = {etype: idx for idx, etype in enumerate(unique_gender_types)}

#Map the genders to numeric values in the DataFrame
df_emp['GenderMapped'] = df_emp['GenderCode'].map(gender_type_mapping)

In [103]:
#Map races to numbers
#Extract unique races types from value_counts
unique_race_types = df_emp['RaceDesc'].value_counts().index.tolist()

#Create the mapping
race_type_mapping = {etype: idx for idx, etype in enumerate(unique_race_types)}

#Map the races to numeric values in the DataFrame
df_emp['RaceMapped'] = df_emp['RaceDesc'].map(race_type_mapping)

In [104]:
#Convert performance score to numbers
#Set a seed for reproducibility
np.random.seed(123)

#Define a mapping function
def convert_performance_to_score(category):
    if category == "Exceeds":
        #return 100
        return np.random.randint(70, 100)  #Random float between 80 and 100
    elif category == "Fully Meets":
        #return 80
        return np.random.randint(50, 70)  #Random float between 50 and 80
    elif category == "Needs Improvement":
        #return 50
        return np.random.randint(20, 50)  #Random float between 20 and 50
    elif category == "PIP":
        #return 25
        return np.random.randint(0, 20)  #Random float between 0 and 20
    else:
        return np.nan  #Handle unexpected values

#Use on performance score
df_emp['PerformanceScoreMapped'] = df_emp['Performance Score'].apply(convert_performance_to_score)


In [105]:
#Infer skill groups based on job titles
def infer_skills(title):
    skills = {
        'PythonCode': 0,
        'JavaScriptCode': 0,
        'SocialMediaSkill': 0,
        'CADSoftwareSkill': 0,
        'NetworkDesign': 0
    }

    #Inferring skills based on job titles (Python, JavaScript, Network Design, and CAD)
    if 'Data Analyst' in title or 'BI Developer' in title or 'Database Administrator' in title:
        skills['PythonCode'] = 1
        skills['JavaScriptCode'] = 1
    if 'Software Engineer' in title or 'Developer' in title:
        skills['PythonCode'] = 1
    if 'Network Engineer' in title or 'IT Manager' in title or 'Infrastructure' in title:
        skills['NetworkDesign'] = 1
    if 'Production Technician' in title or 'CAD' in title:
        skills['CADSoftwareSkill'] = 1
    if 'Sales Manager' in title or 'Social Media' in title:
        skills['SocialMediaSkill'] = 1

    return skills

#Apply skill inference to each job title
skill_columns = ['PythonCode', 'JavaScriptCode', 'SocialMediaSkill', 'CADSoftwareSkill', 'NetworkDesign']
df_emp[skill_columns] = df_emp['Title'].apply(lambda x: pd.Series(infer_skills(x)))


In [106]:
#List of columns to drop
columns_to_drop = [
    "EmpID", "FirstName", "LastName", "StartDate", "ExitDate", "Title", 
    "Supervisor", "ADEmail", "BusinessUnit", "EmployeeStatus", 
    "EmployeeType", "PayZone", "EmployeeClassificationType", 
    "TerminationType", "TerminationDescription", "DepartmentType", 
    "Division", "DOB", "State", "JobFunctionDescription", "GenderCode", 
    "LocationCode", "RaceDesc", "MaritalDesc", "Current Employee Rating", 
    "Education", "Performance Score", "BusinessUnitMapped", "EmployeeClassTypeMapped", 
    "TerminationMapped", "DivisionMapped"
]

#Drop the specified columns
df_emp = df_emp.drop(columns=columns_to_drop, errors='ignore')


In [107]:
df_emp.head()

,YearsWorked,TitleMapped,EducationMapped,EmployeeStatusMapped,EmployeeTypeMapped,PayZoneMapped,DepartmentMapped,Age,GenderMapped,RaceMapped,PerformanceScoreMapped,PythonCode,JavaScriptCode,SocialMediaSkill,CADSoftwareSkill,NetworkDesign
0,5.216438,0,0,0,1,51788.151136,0,55,0,2,63,0,0,0,1,0
1,1.819178,0,0,0,1,150029.753447,0,59,1,4,52,0,0,0,1,0
2,5.994521,2,0,0,0,83611.087214,2,33,1,4,52,0,0,1,0,0
3,3.463014,2,0,0,1,168592.033836,2,26,1,3,56,0,0,1,0,0
4,5.443836,2,0,0,1,180362.827885,2,55,0,3,67,0,0,1,0,0


In [108]:
df_emp.to_csv("clean_emp.csv", index=False)

## Part 2: Data Preprocessing for Job Data

In [22]:
#Find minimum and maximum number of years of experience for the job 
#Extract Min_Exp and Max_Exp using string operations
df_job['Min_Exp'] = df_job['Experience'].str.extract(r'(\d+)', expand=False).astype(int)
df_job['Max_Exp'] = df_job['Experience'].str.extract(r'to (\d+)', expand=False).astype(int)

In [23]:
#Encode education level similar to candidate date
#Define the mapping function
def map_qualifications(qualification):
    if qualification.startswith("B"):
        return 0
    elif qualification.startswith("M"):
        return 1
    elif qualification.startswith("P"):
        return 2
    else:
        return -1  # Handle unexpected values if any

#Apply the mapping to the Qualifications column
df_job['QualificationMapped'] = df_job['Qualifications'].apply(map_qualifications)


In [24]:
#Find minimum and maximum salary values
#Extract Min_Salary and Max_Salary
df_job['Min_Salary'] = df_job['Salary Range'].str.extract(r'\$(\d+)K').astype(int) * 1000
df_job['Max_Salary'] = df_job['Salary Range'].str.extract(r'-\$(\d+)K').astype(int) * 1000

In [25]:
#Replace 'Intern' and 'Temporary' with 'Part-Time' in the 'Work Type' column
df_job['Work Type'] = df_job['Work Type'].replace(['Intern', 'Temporary'], 'Part-Time')

#Display value counts to verify
print(df_job['Work Type'].value_counts())

Work Type
Part-Time    970573
Contract     323131
Full-Time    322236
Name: count, dtype: int64


In [26]:
#Define work type mapping dictionary
work_type_mapping = {'Full-Time': 0, 'Contract': 1, 'Part-Time': 2}

#Map the values in the 'Work Type' column
df_job['WorkMapped'] = df_job['Work Type'].map(work_type_mapping)

In [27]:
#List of job titles to filter
job_titles_to_include = [
    'UX/UI Designer', 'Digital Marketing Specialist', 
    'Software Engineer', 'Network Engineer', 
    'Software Tester', 'Data Scientist', 'Data Analyst'
]

#Filter df_job to include only the specified job titles
#We are interested in only few hypothetical job listings aligned with our candidate base
filtered_df_job = df_job[df_job['Job Title'].isin(job_titles_to_include)].reset_index(drop=True)


In [28]:
#Define skill groups for df_job based on job titles
def infer_job_skills(title):
    skills = {
        'PythonCode': 0,
        'JavaScriptCode': 0,
        'SocialMediaSkill': 0,
        'CADSoftwareSkill': 0,
        'NetworkDesign': 0
    }

    #Infer skills based on job titles
    if 'Data Scientist' in title or 'Data Analyst' in title:
        skills['PythonCode'] = 1
    if 'Software Engineer' in title or 'Software Tester' in title:
        skills['PythonCode'] = 1
        skills['JavaScriptCode'] = 1
    if 'UX/UI Designer' in title:
        skills['JavaScriptCode'] = 1
    if 'Digital Marketing Specialist' in title:
        skills['SocialMediaSkill'] = 1
    if 'Network Engineer' in title:
        skills['NetworkDesign'] = 1

    return skills

#Apply skill inference to each job title in df_job
filtered_df_job[skill_columns] = filtered_df_job['Job Title'].apply(lambda x: pd.Series(infer_job_skills(x)))


In [29]:
#Drop columns
columns_to_drop = [
    "Job Id", "Experience", "Qualifications", "Salary Range", "location", 
    "Country", "latitude", "longitude", "Work Type", "Company Size", 
    "Job Posting Date", "Preference", "Contact Person", "Contact", 
    "Job Title", "Role", "Job Portal", "Job Description", "Benefits", 
    "skills", "Responsibilities", "Company", "Company Profile"
]

#Drop the specified columns
df_job_final = filtered_df_job.drop(columns=columns_to_drop, errors='ignore')


In [30]:
df_job_final.head()

,Min_Exp,Max_Exp,QualificationMapped,Min_Salary,Max_Salary,WorkMapped,PythonCode,JavaScriptCode,SocialMediaSkill,CADSoftwareSkill,NetworkDesign
0,5,15,1,59000,99000,2,0,0,1,0,0
1,4,11,2,65000,91000,0,0,0,0,0,1
2,4,12,1,59000,93000,0,1,1,0,0,0
3,2,8,1,65000,102000,1,0,1,0,0,0
4,2,9,0,65000,102000,2,0,1,0,0,0


In [31]:
df_job_final.to_csv("clean_jobs.csv", index=False)